# Importações

In [3]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Visualização do HOG em tempo real

## Visualização original

In [ ]:
import cv2
import numpy as np
from skimage.draw import line_aa

# --- Configurações ---
cell_size = 8
n_bins = 9  # HOG padrão usa 9 bins de orientação (0 a 180 graus)

def get_hog_histogram(magnitude, angle, cell_size, n_bins):
    """
    Calcula o histograma de orientações para cada célula da imagem.
    """
    H, W = magnitude.shape
    n_cells_y = H // cell_size
    n_cells_x = W // cell_size
    
    # Prepara o array para armazenar os bins: [celulas_y, celulas_x, bins]
    hog_cells = np.zeros((n_cells_y, n_cells_x, n_bins), dtype=float)
    
    # Define o tamanho do bin em graus (180 / 9 = 20 graus)
    angle_unit = 180 / n_bins
    
    for y in range(n_cells_y):
        for x in range(n_cells_x):
            # Recorta a célula atual
            y_start, x_start = y * cell_size, x * cell_size
            cell_mag = magnitude[y_start : y_start + cell_size, x_start : x_start + cell_size]
            cell_ang = angle[y_start : y_start + cell_size, x_start : x_start + cell_size]
            
            # Converte angulos da célula para índices de bin
            # Angulos já estão em graus (0-180). Dividimos por 20 para achar o índice (0-8)
            bin_indices = (cell_ang / angle_unit).astype(int)
            bin_indices[bin_indices >= n_bins] = 0  # Garante que 180 graus caia no bin 0
            
            # Soma as magnitudes para cada bin usando bincount (muito rápido)
            # Flatten transforma a matriz 2D da célula em 1D para o bincount funcionar
            bincount = np.bincount(bin_indices.ravel(), weights=cell_mag.ravel(), minlength=n_bins)
            
            hog_cells[y, x, :] = bincount
            
    return hog_cells

def view_hog_cells(hog_cells, cell_size, frame, img_shape):
    """
    Gera a imagem de visualização baseada nos histogramas calculados.
    (Versão otimizada do seu código fornecido)
    """
    H, W = img_shape
    n_cells_y, n_cells_x, n_bins = hog_cells.shape
    
    # Cria imagem preta de fundo
    viz_image = (frame.copy()).astype('float64')
    
    # Ângulos centrais de cada bin (em radianos para desenhar)
    # 0 a pi (180 graus)
    angles = np.linspace(0, np.pi, n_bins, endpoint=False) + (np.pi / (2 * n_bins))
    
    radius = int(cell_size / 2)
    
    # Normalização global para melhor visualização (opcional, mas ajuda no contraste)
    max_mag = hog_cells.max()
    if max_mag > 0:
        hog_cells /= max_mag

    # --- Loop de desenho ---
    # Nota: Loops em Python puros são lentos para vídeo HD. 
    # Se ficar lento, aumente o cell_size para 16 ou reduza a resolução.
    for i in range(n_cells_y):
        for j in range(n_cells_x):
            
            cx = j * cell_size + cell_size // 2
            cy = i * cell_size + cell_size // 2
            
            for b in range(n_bins):
                strength = hog_cells[i, j, b]
                
                # Só desenha se houver relevância (threshold visual)
                if strength < 0.05: continue 
                
                angle = angles[b]
                
                dx = radius * np.cos(angle)
                dy = radius * np.sin(angle)
                
                r0 = int(round(cy - dy))
                c0 = int(round(cx - dx))
                r1 = int(round(cy + dy))
                c1 = int(round(cx + dx))
                
                # line_aa desenha com anti-aliasing
                rr, cc, val = line_aa(r0, c0, r1, c1)
                
                # Checagem de limites
                valid = (rr >= 0) & (rr < H) & (cc >= 0) & (cc < W)
                
                # Acumula o brilho na imagem final
                viz_image[rr[valid], cc[valid]] += val[valid] * strength

    # Normaliza a imagem final para 0-1 e depois converte para 0-255
    if viz_image.max() > 0:
        viz_image /= viz_image.max()
    
    viz_image = (viz_image * 255).astype(np.uint8)
    return viz_image

# --- Main Loop ---

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Erro: Não foi possível acessar a webcam.")

print("Pressione 'q' para sair")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Reduzi um pouco a resolução para garantir FPS aceitável, 
    # pois desenhar milhares de linhas com line_aa em Python é pesado.
    # Pode voltar para 1280x720 se sua CPU for forte.
    frame = cv2.resize(frame, (1280, 720)) 
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    H, W = gray.shape

    # 1. Calcular Gradientes
    gx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    gy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

    # 2. Magnitude e Fase (Ângulo)
    magnitude = cv2.magnitude(gx, gy)
    angle = cv2.phase(gx, gy, angleInDegrees=True)
    
    # HOG é "unsigned" (0-180), então se for > 180 subtraímos 180, ou usamos módulo.
    angle = np.mod(angle, 180)

    # 3. Calcular Histograma de Células
    hog_cells = get_hog_histogram(magnitude, angle, cell_size, n_bins)

    # 4. Gerar Visualização
    hog_image = view_hog_cells(hog_cells, cell_size, gray, (H, W))

    # Opcional: Mostrar lado a lado (Original P&B vs HOG)
    # Convertendo hog_image para 3 canais para concatenar se quiser, ou mostrar separado
    cv2.imshow('HOG Visualization (Skimage Style)', hog_image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Processando... (Pressione 'q' para sair)


In [ ]:
import cv2
import numpy as np
from skimage.draw import line_aa

# --- Configurações ---
cell_size = 8
magnitude_threshold = 10 # Mantendo seu threshold original
n_bins = 9 

def get_hog_histogram(magnitude, angle, cell_size, n_bins):
    H, W = magnitude.shape
    n_cells_y = H // cell_size
    n_cells_x = W // cell_size
    
    hog_cells = np.zeros((n_cells_y, n_cells_x, n_bins), dtype=float)
    angle_unit = 180 / n_bins
    
    for y in range(n_cells_y):
        for x in range(n_cells_x):
            y_start, x_start = y * cell_size, x * cell_size
            cell_mag = magnitude[y_start : y_start + cell_size, x_start : x_start + cell_size]
            cell_ang = angle[y_start : y_start + cell_size, x_start : x_start + cell_size]
            
            bin_indices = (cell_ang / angle_unit).astype(int)
            bin_indices[bin_indices >= n_bins] = 0
            
            bincount = np.bincount(bin_indices.ravel(), weights=cell_mag.ravel(), minlength=n_bins)
            hog_cells[y, x, :] = bincount
            
    return hog_cells

def view_hog_overlay(frame, hog_cells, cell_size):
    H, W = frame.shape[:2]
    n_cells_y, n_cells_x, n_bins = hog_cells.shape
    
    # Camada temporária para desenhar as linhas (float para acumular brilho)
    hog_layer = np.zeros((H, W), dtype=float)
    
    angles = np.linspace(0, np.pi, n_bins, endpoint=False) + (np.pi / (2 * n_bins))
    radius = int(cell_size / 2)
    
    # Normaliza as células para controlar a intensidade do desenho
    max_val = hog_cells.max()
    if max_val > 0:
        hog_cells /= max_val
        
    for i in range(n_cells_y):
        for j in range(n_cells_x):
            
            cx = j * cell_size + cell_size // 2
            cy = i * cell_size + cell_size // 2
            
            for b in range(n_bins):
                strength = hog_cells[i, j, b]
                
                # Pequeno filtro para não desenhar ruído (opcional, pode remover se quiser ver tudo)
                if strength < 0.05: continue 
                
                angle = angles[b]
                
                dx = radius * np.cos(angle)
                dy = radius * np.sin(angle)
                
                r0 = int(round(cy - dy))
                c0 = int(round(cx - dx))
                r1 = int(round(cy + dy))
                c1 = int(round(cx + dx))
                
                rr, cc, val = line_aa(r0, c0, r1, c1)
                
                valid = (rr >= 0) & (rr < H) & (cc >= 0) & (cc < W)
                
                # Acumula na camada transparente
                hog_layer[rr[valid], cc[valid]] += val[valid] * strength

    # Normaliza a camada HOG para 0-255
    if hog_layer.max() > 0:
        hog_layer /= hog_layer.max()
    hog_layer = (hog_layer * 255).astype(np.uint8)
    
    # Converte a camada de linhas (que é tons de cinza) para 3 canais (BGR)
    # Assim podemos somar com o frame colorido
    hog_layer_color = cv2.merge([hog_layer, hog_layer, hog_layer])
    
    # Soma as imagens: Frame Original + Linhas HOG
    # cv2.add garante que o pixel não "dê a volta" se passar de 255 (satura no branco)
    final_image = cv2.add(frame, hog_layer_color)
        
    return final_image

# --- Loop Principal ---

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Erro: Não foi possível acessar a webcam.")

print("Pressione 'q' para sair")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_invertido = cv2.flip(frame, 1)
    # Resize (mantendo o que foi discutido sobre performance, ajuste se necessário)q
    frame = cv2.resize(frame_invertido, (1280, 720))
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Cálculos HOG
    gx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    gy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

    magnitude = cv2.magnitude(gx, gy)
    angle = cv2.phase(gx, gy, angleInDegrees=True)
    angle = np.mod(angle, 180)

    hog_cells = get_hog_histogram(magnitude, angle, cell_size, n_bins)

    # Gera a visualização SOBRE o frame original
    frame_com_hog = view_hog_overlay(frame, hog_cells, cell_size)

    cv2.imshow('HOG Real-Time Overlay', frame_com_hog)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Pressione 'q' para sair


## Visualização com setas

In [ ]:
cap = cv2.VideoCapture(0) 

cell_size = 8 
magnitude_threshold = 10

scale_factor = 0.1

if not cap.isOpened():
    print("Erro: Não foi possível acessar a webcam.")

print("Pressione 'q' para sair")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (1280, 720))
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    H, W = gray.shape

    gx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    gy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

    magnitude = cv2.magnitude(gx, gy)

    for y in range(0, H, cell_size):
        for x in range(0, W, cell_size):
            
            cell_mag = magnitude[y:y+cell_size, x:x+cell_size]
            cell_gx  = gx[y:y+cell_size, x:x+cell_size]
            cell_gy  = gy[y:y+cell_size, x:x+cell_size]

            avg_mag = np.mean(cell_mag)
            
            if avg_mag > magnitude_threshold:
                
                avg_gx = np.mean(cell_gx)
                avg_gy = np.mean(cell_gy)
                
                avg_ang = np.arctan2(avg_gy, avg_gx)

                center_x = x + cell_size // 2
                center_y = y + cell_size // 2

                arrow_length = avg_mag * scale_factor
                
                if arrow_length > cell_size * 2:
                    arrow_length = cell_size * 2

                end_x = int(center_x + arrow_length * np.cos(avg_ang))
                end_y = int(center_y + arrow_length * np.sin(avg_ang))

                cv2.arrowedLine(frame, (center_x, center_y), (end_x, end_y), (0, 0, 255), 1, tipLength=0.3)

    cv2.imshow('HOG - Visualização em tempo real (Corrigida)', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Pressione 'q' para sair


In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Erro: Webcam não encontrada!")
    exit()

cell_size = 8
magnitude_threshold = 10
scale_factor = 0.1

print("Pressione 'q' para sair")

while True:
    ret, frame = cap.read()
    if not ret or frame is None:
        print("Falha ao capturar frame.")
        break

    # frame = cv2.resize(frame, (1280, 720))

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    H, W = gray.shape

    gx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    gy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

    magnitude = cv2.magnitude(gx, gy)

    for y in range(0, H, cell_size):
        for x in range(0, W, cell_size):

            cell_mag = magnitude[y:y+cell_size, x:x+cell_size]
            cell_gx  = gx[y:y+cell_size, x:x+cell_size]
            cell_gy  = gy[y:y+cell_size, x:x+cell_size]

            avg_mag = np.mean(cell_mag)

            if avg_mag > magnitude_threshold:
                avg_gx = np.mean(cell_gx)
                avg_gy = np.mean(cell_gy)
                avg_ang = np.arctan2(avg_gy, avg_gx)

                cx = x + cell_size // 2
                cy = y + cell_size // 2

                arrow_len = min(avg_mag * scale_factor, cell_size * 2)

                ex = int(cx + arrow_len * np.cos(avg_ang))
                ey = int(cy + arrow_len * np.sin(avg_ang))

                cv2.arrowedLine(frame, (cx, cy), (ex, ey), (0, 0, 255), 1, tipLength=0.3)

    cv2.imshow('HOG - Visualização em tempo real', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Pressione 'q' para sair
